## Get `Intent`

In [4]:
import pickle
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from keras.models import load_model
from joblib import load


In [5]:
ml_model = load('SVC_Model.joblib')

def ml_predict(sentence):
    pred = ml_model.predict([sentence])[0]
    return {'Intent':pred}

In [6]:
dl_model = load_model('BOW.h5')

words = pickle.load(open("..\data\words.pkl",'rb'))
classes = pickle.load(open("..\data\classes.pkl",'rb'))
token = Tokenizer(num_words=(len(words)+1))
token.fit_on_texts(words)

# define a function to clean the text
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemma.lemmatize(word.lower()) for word in sentence_words if word not in stopwords.words('english')]
    return sentence_words

def dl_predict(sentence):
    
    # filter predictions below a threshold
    bow = token.texts_to_matrix([clean_up_sentence(sentence)]) 
    res = dl_model.predict(bow)[0]
    results = [[i,r] for i,r in enumerate(res) if r>.50]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = {"Intent":classes[r[0]] for r in results}
    return return_list


In [4]:
sentences = ['open the door please' , 'how is the weather today' ,
             'please play music by the beatles', 
             'please play the song I like','hi there']
for sentence in sentences:
    print(dl_predict(sentence))

{'Intent': 'Iot'}
{'Intent': 'Weather'}
{'Intent': 'PlayMusic'}
{'Intent': 'PlayMusic'}
{'Intent': 'Greeting'}


In [5]:
sentences = ['open the door please' , 'how is the weather today' ,
             'please play music by the beatles',
             'please play the song I like','hi there']
for sentence in sentences:
    print(ml_predict(sentence))

{'Intent': 'Iot'}
{'Intent': 'Weather'}
{'Intent': 'PlayMusic'}
{'Intent': 'PlayMusic'}
{'Intent': 'Greeting'}


-----------------------------------------------------------

## Get `Entities`

In [7]:
nlp = spacy.load(r'..\Name Entity Recognition')

def NER(sentence):
    doc = nlp(sentence)
    result = {ent.label_: ent.text for ent in doc.ents}        
    return result
    

In [7]:
NER('What is the weather in berlin?')

{'location': 'berlin'}

In [8]:
NER('play music for Cairokee') 

{'artist': 'Cairokee'}

In [9]:
NER('dim lights in bedroom')

{'action': 'dim', 'device': 'lights', 'room': 'bedroom'}

In [10]:
NER('play insan by hamza namira')

{'song_name': 'insan', 'artist': 'hamza namira'}

In [11]:
NER('what is the weather')

{}

------------------------------


## Final Function That Return `NLU` For Utterance

In [8]:
def NLU(sentence):
    intent = dl_predict(sentence)
    entity = NER(sentence)
    
    if entity == {}:
        result = {'Intent': intent['Intent'],'Entities':None}
    else:
        result = {'Intent': intent['Intent'],'Entities':entity}
        
    return result
    

In [20]:
NLU('set an alarm for 7 am')

{'Intent': 'Reminder', 'Entities': {'time': '7 am'}}

In [14]:
NLU('tell me how is the weather today')

{'Intent': 'Weather', 'Entities': {'date': 'today'}}

In [15]:
NLU('dim lights in bathroom')

{'Intent': 'Iot',
 'Entities': {'action': 'dim', 'device': 'lights', 'room': 'bathroom'}}

In [16]:
NLU('hoe is the weather tomorrow?')

{'Intent': 'Weather', 'Entities': {'date': 'tomorrow'}}

In [17]:
NLU('what is the weather')

{'Intent': 'Weather', 'Entities': None}

In [18]:
NLU('turn the lights off in bedroom')

{'Intent': 'Iot',
 'Entities': {'device': 'lights', 'action': 'off', 'room': 'bedroom'}}

In [19]:
NLU('play shallow by lady gaga')

{'Intent': 'PlayMusic',
 'Entities': {'song_name': 'shallow', 'artist': 'lady gaga'}}

In [10]:
NLU('Hey leo how is the weather today')


{'Intent': 'Weather', 'Entities': {'date': 'today'}}